In [71]:
import os
import time
import requests
import pandas as pd
import numpy as np

In [85]:
api_key = "x"
username = "Friskis1"
MatchNumber = "50"

In [ ]:
api_AccountURL = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + username
print(api_AccountURL)

api_AccountURLandAPI = api_AccountURL + '?api_key=' + api_key
print(api_AccountURLandAPI)

responseAccount = requests.get(api_AccountURLandAPI)
player_info = responseAccount.json()
# print(player_info)

puuid = player_info['puuid']
# print(puuid)

api_MatchHistory = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuid + "/ids?start=0&count=" + MatchNumber
# print(api_MatchHistory)

api_MatchHistoryAPI = api_MatchHistory + '&api_key=' + api_key 
# print(api_MatchHistoryAPI)

responseMatchHistory = requests.get(api_MatchHistoryAPI)
match_ids = responseMatchHistory.json()

print(match_ids)

time.sleep(120)
print("execute other")

In [4]:
# def calculate_combined_gold(participants):
#     total_gold = 0
#     for participant in participants:
#         total_gold += participant['goldEarned']
#     return total_gold

In [46]:
# for i in range(int(MatchNumber)):
    # api_MatchData = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_ids[i] + '?api_key=' + api_key
    # print(api_MatchData)

    # responseMatchData = requests.get(api_MatchData)
    # match_data = responseMatchData.json()

    # Calculate combined gold of the first 5 players
    # team1_participants = match_data['info']['participants'][:5]
    # team1_gold = calculate_combined_gold(team1_participants)
    # print("Combined gold of Blue side players:", team1_gold)

    # Calculate combined gold of the last 5 players
    # team2_participants = match_data['info']['participants'][-5:]
    # team2_gold = calculate_combined_gold(team2_participants)
    # print("Combined gold of Red side players:", team2_gold)
    
    # print("Blue side won:", match_data['info']['participants'][0]['win'])

    # if match_data['info']['participants'][0]['win']:
        # winning_team_gold = team1_gold
        # lossing_team_gold = team2_gold
    # else:
        # winning_team_gold = team2_gold
        # lossing_team_gold = team1_gold

    # print("Combined gold winning team:", winning_team_gold)
    # print("Combined gold lossing team:", lossing_team_gold)
    # print("Gold difference between winning and lossing:", winning_team_gold - lossing_team_gold)

    # print("")

In [41]:
# team_1["Gold"]

In [21]:
# api_MatchTimeline = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_ids[0] + '/timeline' + '?api_key=' + api_key
# responseMatchTimeline = requests.get(api_MatchTimeline)
# match_timeline = responseMatchTimeline.json()

In [42]:
# match_timeline["info"]["frames"][14]["participantFrames"][f"{j}"]["totalGold"]

In [87]:
file_path = "team_data.csv"

if os.path.exists(file_path):
    os.remove(file_path)

combined_data = []  # List to store the combined data for all iterations

for i in range(int(MatchNumber)):
    api_MatchData = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_ids[i] + '?api_key=' + api_key
    responseMatchData = requests.get(api_MatchData)
    match_data = responseMatchData.json()

    api_MatchTimeline = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_ids[i] + '/timeline' + '?api_key=' + api_key
    responseMatchTimeline = requests.get(api_MatchTimeline)
    match_timeline = responseMatchTimeline.json()
    
    if match_data["info"]["gameMode"] == "CLASSIC":

        team_1 = {"Gold": 0, "Level": 0, "Minions": 0, "Jungle_minions": 0}
        team_2 = {"Gold": 0, "Level": 0, "Minions": 0, "Jungle_minions": 0}
    
        frame_index = min(14, len(match_timeline["info"]["frames"]) - 1)

        for j in range(1, 6):
            participant_frame = match_timeline["info"]["frames"][frame_index]["participantFrames"].get(str(j))
            if participant_frame is not None:
                team_1["Gold"] += participant_frame["totalGold"]
                team_1["Level"] += participant_frame["level"]
                team_1["Minions"] += participant_frame["minionsKilled"]
                team_1["Jungle_minions"] += participant_frame["jungleMinionsKilled"]

        for j in range(6, 11):
            participant_frame = match_timeline["info"]["frames"][frame_index]["participantFrames"].get(str(j))
            if participant_frame is not None:
                team_2["Gold"] += participant_frame["totalGold"]
                team_2["Level"] += participant_frame["level"]
                team_2["Minions"] += participant_frame["minionsKilled"]
                team_2["Jungle_minions"] += participant_frame["jungleMinionsKilled"]
            
        team_1["Level"] /= 5
        team_2["Level"] /= 5

        team_1["Gold_diff"] = team_1["Gold"] - team_2["Gold"]
        team_2["Gold_diff"] = team_2["Gold"] - team_1["Gold"]

        if match_data["info"]["teams"][0]["win"]:
            team_1["Win"] = 1
            team_2["Win"] = 0
        else:
            team_1["Win"] = 0
            team_2["Win"] = 1

        team_1_data = {'Gold': team_1["Gold"], 'Level': team_1["Level"], 'Minions': team_1["Minions"], 'Jungle_minions': team_1["Jungle_minions"], 'Gold_diff': team_1["Gold_diff"], 'Win': team_1["Win"]}
        team_2_data = {'Gold': team_2["Gold"], 'Level': team_2["Level"], 'Minions': team_2["Minions"], 'Jungle_minions': team_2["Jungle_minions"], 'Gold_diff': team_2["Gold_diff"], 'Win': team_2["Win"]}

        combined_data.append(team_1_data)
        combined_data.append(team_2_data)

combined_df = pd.DataFrame(combined_data)
combined_df.to_csv('team_data.csv', index=False)

KeyError: 'info'